In [5]:
!pip3 install tensorflow-gpu==2.0.0-beta0
#!pip install tensorflow-transform

  Using cached https://files.pythonhosted.org/packages/e8/7e/87c4c94686cda7066f52cbca4c344248516490acdd6b258ec6b8a805d956/tensorflow_gpu-2.0.0b0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: tensorflow-gpu 1.15.0
    Uninstalling tensorflow-gpu-1.15.0:
      Successfully uninstalled tensorflow-gpu-1.15.0


Mounting the drive and extracting the HAPT Dataset

In [3]:
from google.colab import drive
drive.mount ('/content/gdrive' )
root_path = '/content/gdrive/My Drive/Dataset/HAPT Data Set/'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Importing of the packages

In [4]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
def generate_acc_filename(expID, userID):
  if expID <= 9:
    filename = "/acc_exp0"+ str(expID)
  else:
    filename = "/acc_exp"+ str(expID)
  if userID <= 9:
    filename = filename + "_user0"+str(userID)+ ".txt"
  else:
    filename = filename + "_user"+str(userID)+ ".txt"
  return filename

def generate_gyro_filename(expID, userID):
  if expID <= 9:
    filename = "/gyro_exp0"+ str(expID)
  else:
    filename = "/gyro_exp"+ str(expID)
  if userID <= 9:
    filename = filename + "_user0"+str(userID)+ ".txt"
  else:
    filename = filename + "_user"+str(userID)+ ".txt"
  return filename

In [7]:
#Defining the path location
training_dataset_path = root_path + "/Training"
testing_dataset_path = root_path + "/Test"
validation_dataset_path = root_path + "/Validation"
labels = pathlib.Path(root_path + "/labels.txt")
print("Path of the labels file:",labels)

lables_array = np.loadtxt(fname = labels)
print("The first row in the labels array",lables_array[0])
print("Shape of the labels:", lables_array.shape)

Path of the labels file: /content/gdrive/My Drive/Dataset/HAPT Data Set/labels.txt
The first row in the labels array [1.000e+00 1.000e+00 5.000e+00 2.500e+02 1.232e+03]
Shape of the labels: (1214, 5)


Set the location of the root, training, test and validation datasets

In [0]:
def prepare_input_array(filepath, StartLimit, EndLimit):
  input_array = np.empty((0, 7))
  for x in lables_array:
    expID = int(x[0])
    userID = int(x[1])
    label = int(x[2])
    start_time = int(x[3])
    End_time = int(x[4])
    if userID >= StartLimit and userID <= EndLimit:
      accpath = pathlib.Path(filepath+generate_acc_filename(expID, userID))
      #print(accpath)
      features_acc = (np.loadtxt(fname = accpath))[start_time-1:End_time]
      #print(features_acc.shape)
      gyropath = pathlib.Path(filepath+generate_gyro_filename(expID, userID))
      #print(gyropath)
      features_gyro = (np.loadtxt(fname = gyropath))[start_time-1:End_time]     
      #print(features_gyro.shape)
      rows,columns = features_gyro.shape
      features = np.hstack((features_acc, features_gyro))
      #print("Total Features:", features.shape)
      labels = np.full((rows,1),label)
      #print("labels:", labels.shape)
      features_labels = np.hstack((features,labels))
      #print("Features with lables:",features_labels.shape )
      #print(type(features_labels))
      #print(features_labels)
      input_array = np.append(input_array, features_labels, axis = 0)
      #print(type(features_labels))
      #print(features_labels)
  return input_array[:,[0,5]],input_array[:,[6]]

In [0]:
train_feature, train_label = prepare_input_array(training_dataset_path, 1, 21)
validation_feature, validation_label = prepare_input_array(validation_dataset_path, 28, 30)
test_feature, test_label = prepare_input_array(testing_dataset_path, 22, 27)

In [22]:
print(train_feature.shape, train_label.shape)
print(validation_feature.shape, validation_label.shape)
print(test_feature.shape, test_label.shape)
print(test_feature)
print(test_label)

(554294, 2) (554294, 1)
(86018, 2) (86018, 1)
(175302, 2) (175302, 1)
[[1.01111115 0.01985312]
 [1.0194445  0.04764749]
 [1.02916675 0.05131268]
 ...
 [1.05277781 0.18753563]
 [1.36388891 0.20097467]
 [1.45416669 0.22510384]]
[[5.]
 [5.]
 [5.]
 ...
 [2.]
 [2.]
 [2.]]


In [0]:
'''def normalization(input_arr):
  for x in range(6):
    feature_column = input_array[:x]
    normalized_feature_column = tft.scale_to_z_score(feature_column)
    print(normalized_feature_column)
normalization(test_array)'''

In [36]:
#creating tensorflow dataset objects
train_ds = tf.data.Dataset.from_tensor_slices((train_feature,train_label))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_feature,validation_label))
test_ds = tf.data.Dataset.from_tensor_slices((test_feature,test_label))

print(train_ds)
#windowing and batching
train_ds = train_ds.window(250,125)
validation_ds = validation_ds.window(250,125)
test_ds = test_ds.window(250,125)

<TensorSliceDataset shapes: ((2,), (1,)), types: (tf.float64, tf.float64)>
